# Day 1: The setup with kaggle mnist dataset

In [2]:
import kaggle
import keras
import torch
%load_ext tensorboard
# check for gpu
!nvidia-smi
# check for cuda
!nvcc --version
# use gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Sat May 18 11:14:45 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0  On |                  Off |
|  0%   41C    P8             18W /  450W |     727MiB /  24564MiB |     15%      Default |
|                                         |       

Purpose of this notebook is to:
* run kaggle mnist locally
* on gpu
* and upload the results to kaggle.

##  Download the dataset from kaggle

In [3]:
import os
import zipfile

# Define the directory where you want to download the data
data_dir = "MyDataset/mnist/raw"  # './' represents the current directory

# Check if the directory exists
if not os.path.exists(data_dir):
    # If not, create the directory
    os.makedirs(data_dir)

# Move to that directory
os.chdir(data_dir)
competition_name = "digit-recognizer"
# Download the data
os.system("kaggle competitions download -c " + competition_name)

# Unzip the data
with zipfile.ZipFile("digit-recognizer.zip","r") as zip_ref:
    zip_ref.extractall(".")

os.chdir("../../..")

digit-recognizer.zip: Skipping, found more recently modified local copy (use --force to force download)


## Load Data And prepare the data

In [4]:
import pandas as pd
import numpy as np

# Load the dataset
train_df = pd.read_csv('MyDataset/mnist/raw/train.csv')
test_df = pd.read_csv('MyDataset/mnist/raw/test.csv')

# Split features and labels
y_train = train_df["label"]
x_train = train_df.drop(labels = ["label"], axis = 1)

# Convert to numpy arrays
x_train = x_train.values
y_train = y_train.values

# Test data
x_test = test_df.values



### Inspet the data

In [5]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Prepare the data

In [6]:
# Make Validation set
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=2)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
x_val = x_val.reshape(-1, 28, 28, 1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (37800, 28, 28, 1)
y_train shape: (37800,)
37800 train samples
28000 test samples


## Make model

In [7]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

2024-05-18 11:14:49.189506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-18 11:14:49.189603: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-18 11:14:49.189649: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

### Compile the model

In [8]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

### Callbacks   

In [9]:
callbacks = [ 
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=10),
    keras.callbacks.ModelCheckpoint("mnist_model.keras", save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="./logs"),
]
%tensorboard --logdir=logs

### Fit the model

In [10]:
batch_size = 128
epochs = 20

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_val, y_val),

)

score = model.evaluate(x_val, y_val, verbose=0)

Epoch 1/20


I0000 00:00:1716030891.694184    1282 service.cc:145] XLA service 0x7d7e5c0171a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716030891.694205    1282 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-05-18 11:14:51.716276: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-18 11:14:51.803685: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


 85/296 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.2274 - loss: 2.0831

I0000 00:00:1716030893.364720    1282 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


296/296 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - acc: 0.4682 - loss: 1.4758 - val_acc: 0.9388 - val_loss: 24.5115
Epoch 2/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9117 - loss: 0.2929 - val_acc: 0.9626 - val_loss: 16.4309
Epoch 3/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9453 - loss: 0.1797 - val_acc: 0.9681 - val_loss: 13.4111
Epoch 4/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9553 - loss: 0.1544 - val_acc: 0.9593 - val_loss: 21.8934
Epoch 5/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9630 - loss: 0.1263 - val_acc: 0.9745 - val_loss: 10.0660
Epoch 6/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9683 - loss: 0.1078 - val_acc: 0.9817 - val_loss: 8.9775
Epoch 7/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9739 - loss: 0.0935 - val_acc: 0.9779 - val_loss: 12.4254
Epoch 8/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9766 - loss: 0.0832 - val_acc: 0.9831 - val_loss: 9.1010
Epoch 9/20
296/296 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.

In [11]:

import pandas as pd
# Assuming 'predictions' is an array containing your model's predictions
predictions = model.predict(x_test)


875/875 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step


In [12]:
class_predictions = np.argmax(predictions, axis=1)
# Create a DataFrame with the prediction results
# 'ImageId' is a common column name in MNIST-like competitions
submission = pd.DataFrame({
    "ImageId": list(range(1, len(class_predictions) + 1)),
    "Label": class_predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('my_submission.csv', index=False)

In [13]:
#!kaggle competitions submit -c digit-recognizer -f my_submission.csv -m "First submission"


In [14]:
!kaggle competitions submissions digit-recognizer

fileName           date                 description            status    publicScore  privateScore  
-----------------  -------------------  ---------------------  --------  -----------  ------------  
my_submission.csv  2024-05-06 16:32:30  Best model submission  complete  0.99403                    
my_submission.csv  2024-05-04 08:02:41  First submission       complete  0.9926                     
my_submission.csv  2024-05-04 08:02:09  First submission       complete  0.9926                     
